In [8]:
import math
import torch
import pandas as pd
import numpy as np
from utils import toTorchTensers, NormalizeData
from Models import ForcastModel

data_url = r'./data/Data.xlsx'
data_egypt = pd.read_excel(data_url, sheet_name='Egypt')
data_vietnam = pd.read_excel(data_url, sheet_name='Vietnam')

In [9]:
# data
DATA = data_egypt
DATA = NormalizeData(DATA)
DATA = DATA.to_numpy()

np.random.seed(1)
np.random.shuffle(DATA)
split_point = math.floor(len(DATA) * 0.75)

train_x, train_y = DATA[:split_point, 1:], DATA[:split_point, 0:1]
test_x, test_y = DATA[split_point:, 1:], DATA[split_point:, 0:1]

[train_x, train_y, test_x, test_y] = toTorchTensers(
  train_x, train_y, test_x, test_y
)

In [10]:
# training
model = ForcastModel()
model.fit(train_x, train_y, epoches=50)

epoche: 0, loss: 0.00025065, execution time: 1.0s
epoche: 1, loss: 0.00003880, execution time: 1.1s
epoche: 2, loss: 0.00001137, execution time: 1.1s
epoche: 3, loss: 0.00000314, execution time: 1.1s
epoche: 4, loss: 0.00000103, execution time: 1.1s
epoche: 5, loss: 0.00000026, execution time: 1.1s
epoche: 6, loss: 0.00000002, execution time: 1.1s
epoche: 7, loss: 0.00000001, execution time: 1.1s
epoche: 8, loss: 0.00000000, execution time: 1.1s
epoche: 9, loss: 0.00000000, execution time: 1.1s
epoche: 10, loss: 0.00000000, execution time: 1.1s
epoche: 11, loss: 0.00000000, execution time: 1.1s
epoche: 12, loss: 0.00000000, execution time: 1.1s
epoche: 13, loss: 0.00000000, execution time: 1.1s
epoche: 14, loss: 0.00000000, execution time: 1.1s
epoche: 15, loss: 0.00000000, execution time: 1.1s
epoche: 16, loss: 0.00000000, execution time: 1.1s
epoche: 17, loss: 0.00000000, execution time: 1.0s
epoche: 18, loss: 0.00000000, execution time: 1.0s
epoche: 19, loss: 0.00000000, execution t

In [13]:
# testing
model.test(test_x, test_y)
model.test(train_x, train_y)


loss: 0.0000, acc: 0.8919
loss: 0.0000, acc: 0.9322
